In [1]:
# Optimize hidden states at tips given true parameters of generating model.
import ivy
from ivy.chars import hrm
import numpy as np

In [ ]:
# True generating matrix
# (-.11, 0.01, 0.1, 0,
#   0.01, -0.11, 0, 0.1,
#   0.1, 0, -0.7, 0.6,
#   0, 0.1, 0.6, -0.7)

In [12]:
tree = ivy.tree.read("../support/hrm_300tips.newick")
chars = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 
        1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0]
truechars = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 3, 3, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 
            2, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 
            3, 2, 2, 3, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
            0, 0, 0, 0, 2, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 
            2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 
            1, 1, 1, 0, 0, 0, 0, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 
            3, 2, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0]
Q = hrm.fill_Q_matrix(2,2,[0.01, 0.1, 0.01, 0.1, 0.1, 0.6, 0.1,0.6])

In [13]:
out = hrm.hrm_multipass(tree, chars, Q, 2, returntips=True)

In [15]:
count = 0
for i,n in enumerate(out[1]):
    if chars[i] == 0:
        rem = [1,3]
    else:
        rem = [0,2]
    tmp = n[:-1]
    tmp[rem] = 0
    tmp = tmp/sum(tmp)
    
    print tmp, tmp.argmax(), truechars[i]
    if tmp.argmax() != truechars[i]:
        count += 1

[ 0.99752811  0.          0.00247189  0.        ] 0 0
[ 0.99752811  0.          0.00247189  0.        ] 0 0
[  9.99198463e-01   0.00000000e+00   8.01537298e-04   0.00000000e+00] 0 0
[  9.99198463e-01   0.00000000e+00   8.01537298e-04   0.00000000e+00] 0 0
[ 0.99521095  0.          0.00478905  0.        ] 0 0
[ 0.99866387  0.          0.00133613  0.        ] 0 0
[  9.99915924e-01   0.00000000e+00   8.40756873e-05   0.00000000e+00] 0 0
[  9.99915924e-01   0.00000000e+00   8.40756873e-05   0.00000000e+00] 0 0
[  9.99761358e-01   0.00000000e+00   2.38641671e-04   0.00000000e+00] 0 0
[  9.99920017e-01   0.00000000e+00   7.99827499e-05   0.00000000e+00] 0 0
[  9.99719356e-01   0.00000000e+00   2.80643611e-04   0.00000000e+00] 0 0
[ 0.99769609  0.          0.00230391  0.        ] 0 0
[ 0.99769609  0.          0.00230391  0.        ] 0 0
[ 0.99776168  0.          0.00223832  0.        ] 0 0
[  9.99275867e-01   0.00000000e+00   7.24132505e-04   0.00000000e+00] 0 0
[  9.99275867e-01   0.00000000

In [16]:
count

22